# Image Processing SS 20 - Assignment - 05

### Deadline is 27.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import chelsea
from skimage.color import rgb2gray

import random
from io import BytesIO
    
from PIL import Image
import itertools

pylab.rcParams['figure.figsize'] = (12, 8)   # This makes the plot bigger

# Exercise 1 - Filters - 5 Points

Implement and apply the filters below to noisy images. Plot the noisy image vs the filtered one

In [ ]:
img = rgb2gray(chelsea() / 255.)
img.shape

In [ ]:
def salt_peper(img, p):
    peper_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    salt_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    img_noise = img.copy()
    img_noise[peper_mask] = 0
    img_noise[salt_mask] = 1
    return img_noise

salt_peper_noises = [0.01, 0.03, 0.1]
imgs_salt_peper = [salt_peper(img, p) for p in salt_peper_noises]
for img_salt_peper in imgs_salt_peper:
    plt.imshow(img_salt_peper, cmap='gray')
    plt.show()

In [ ]:
def gaussian_noise(img, std):
    return np.clip(img + np.random.normal(0, std, img.shape), 0, 1)

gaussian_noises = (0.05, 0.10, 0.2)
imgs_gaussian_noise = [gaussian_noise(img, s) for s in gaussian_noises]
for img_gaussian in imgs_gaussian_noise:
    plt.imshow(img_gaussian, cmap='gray')
    plt.show()

In [ ]:
# implement the filters:

def arithmetic_mean(img):
    win_size=3
    height, width = img.shape
    edge=int((win_size-1)/2)
    for i in range(edge,height-edge):
        for j in range(edge,width-edge):
            img[i, j] = np.mean(img[i - edge:i + edge + 1, j - edge:j + edge + 1])
    return img

def geometric_mean(img):
    win_size=3
    height, width = img.shape
    edge=int((win_size-1)/2)
    for i in range(edge,height-edge):
        for j in range(edge,width-edge):
            img[i, j] = (np.prod(img[i - edge:i + edge + 1, j - edge:j + edge + 1]))**(1.0/9)
    return img

def contraharmoic_mean(img, q=None):
    # Your need to select the q yourself
    if q==None:
        q=0
    win_size=3
    height, width = img.shape
    edge=int((win_size-1)/2)
    for i in range(edge,height-edge):
        for j in range(edge,width-edge):
            img[i, j] = np.sum(img[i - edge:i + edge + 1, j - edge:j + edge + 1]**(q+1))/np.sum(img[i - edge:i + edge + 1, j - edge:j + edge + 1]**q)
    return img
def AdaptProcess_mean(img, i, j, minSize, maxSize):
    filter_size = minSize
    kernelSize = filter_size // 2
    img_kernel = img[i-kernelSize:i+kernelSize+1, j-kernelSize:j+kernelSize+1]
    minPix = np.min(img_kernel)
    maxPix = np.max(img_kernel)
    meanPix = np.mean(img_kernel)
    zxy = img[i,j]

    if (meanPix > minPix) and (meanPix < maxPix):
        if (zxy > minPix) and (zxy < maxPix):
            return zxy
        else:
            return meanPix
    else:
        filter_size = filter_size + 2
        if filter_size <= maxSize:
            return AdaptProcess_mean(img, i, j, filter_size, maxSize)
        else:
            return meanPix
def adaptive_mean(img,minSize=3,maxSize=7):
    height, width = img.shape
    edge=int((maxSize-1)/2)
    for n in range(edge,height-edge):
        for m in range(edge,width-edge):
            img[n,m] = AdaptProcess_mean(img, n, m, minSize, maxSize)
    return img

def AdaptProcess_median(img, i, j, minSize, maxSize):
    filter_size = minSize
    kernelSize = filter_size // 2
    img_kernel = img[i-kernelSize:i+kernelSize+1, j-kernelSize:j+kernelSize+1]
    minPix = np.min(img_kernel)
    maxPix = np.max(img_kernel)
    medPix = np.median(img_kernel)
    zxy = img[i,j]

    if (medPix > minPix) and (medPix < maxPix):
        if (zxy > minPix) and (zxy < maxPix):
            return zxy
        else:
            return medPix
    else:
        filter_size = filter_size + 2
        if filter_size <= maxSize:
            return AdaptProcess_median(img, i, j, filter_size, maxSize)
        else:
            return medPix
def adaptive_median(img,minSize=3,maxSize=7):
    height, width = img.shape
    edge=int((maxSize-1)/2)
    for n in range(edge,height-edge):
        for m in range(edge,width-edge):
            img[n,m] = AdaptProcess_median(img, n, m, minSize, maxSize)
    return img

In [ ]:
filters = [arithmetic_mean, geometric_mean, contraharmoic_mean, 
           adaptive_mean, adaptive_median]
for filter in filters:
    for sp, img_sp in zip(salt_peper_noises, imgs_salt_peper):
        plt.suptitle(filter.__name__ + ", salt peper noise: {}".format(sp))
        plt.subplot(221)
        plt.imshow(img_sp, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_sp.copy()), cmap='gray')
        plt.show()


In [ ]:
for filter in filters:
    for gn, img_gn in zip(gaussian_noises, imgs_gaussian_noise):
        plt.suptitle(filter.__name__ + ", gaussian noise: {}".format(gn))
        plt.subplot(221)
        plt.imshow(img_gn, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_gn.copy()), cmap='gray')
        plt.show()

# Exercise 2 - Sharpening - 5 Points

Sharpen the image `img_blurry`. You need to implement the `sharp_laplacian` and `unsharp_masking` functions. You might
consider some processing steps before sharpening.

In [ ]:
from skimage.filters import gaussian,laplace

In [ ]:
img = rgb2gray(chelsea() / 255.)
img_blurry = gaussian_noise(gaussian(img , 1.6), 0.07)
plt.imshow(img_blurry, cmap='gray')
np.amax(img_blurry), np.amin(img_blurry)

In [ ]:
import scipy.ndimage.filters as filters
def sharp_laplacian(img):
    """Perform sharpening with a laplacian filter"""
    # your code here
    img=img.copy()
    img=contraharmoic_mean(img,0) #get rid of the noise
    img_ori=img.copy()
    #La_kernel = np.ones((3,3))*(-1)
    #La_kernel[1,1] = 8
    La_kernel=np.array([[0,1,0],[1,-4,1],[0,1,0]])
    la_img=filters.convolve(img,La_kernel)  #convolve with the Laplacian Kernel
    Sharpen_factor=2  #Sharpening factor
    la_img= la_img*Sharpen_factor  #Scale the filter
    out=img_ori-la_img  #Sharpening
    out=np.clip(out,0,1) #constrain the pixel value between 0 and 1
    return out
    

def unsharp_masking(img):
    """Perform sharpening by unsharp masking"""
    # your code here
    a=1  #factor
    img_ori=img.copy()
    img_ori=contraharmoic_mean(img,1)  #get rid of the noise
    img_blur=gaussian(img_ori, 1.6)  #Blur the image
    img_sharp=img_ori-img_blur  # get the unsharp mask
    return img_ori+a*img_sharp #sharpen the image

In [ ]:
plt.imshow(sharp_laplacian(img_blurry), cmap='gray')
plt.show()

In [ ]:
plt.imshow(unsharp_masking(img_blurry), cmap='gray')
plt.show()